# Standard Attributes

Standard attributes are HDF5 attributes that follow **specific rules (conventions)** defined by the user
through `StandardAttribute` classes. These classes are registered for the selected (or multiple) HDF5 objects (File, Group, Dataset).

In [1]:
import h5rdmtoolbox as h5tbx
from h5rdmtoolbox import conventions

Let's define a standard attribute. Here is the summary in short of what the attribute should be like:
* Name of the attribute is `software`.
* User shall provide `name` and `version`
* It should only be available for `core.Dataset` objects
* The HDF attribute string in the end should look be dictionary-like: `{'name': <software-name>, 'vers': <version>}`

We write a subclass of `StandardAttribute`, set the class-property `name` and control the above described behaviour in the `setter` and `getter` methods. We also write a small dataclass `Software` which is the return object to the user:

In [2]:
import json
from dataclasses import dataclass

@dataclass(frozen=True)
class Software:
    name: str
    vers: str

class SoftwareAttribute(conventions.StandardAttribute):
    
    name = 'software'
    
    def setter(self, obj, name_version: "Tuple"):
        name_vers_dict = dict(name=name_version[0], vers=name_version[1])
        obj.attrs.create(self.name, json.dumps(name_vers_dict))
        
    def getter(self, obj):
        sftw_str = self.safe_getter(obj)
        sftw_dict = json.loads(sftw_str)
        return Software(**sftw_dict)

Register the instance (!) of `SoftwareAttribute` to `core.Group` and `core.Dataset`:

In [3]:
conventions.register_standard_attribute(attribute=SoftwareAttribute(),
                                        cls=h5tbx.wrapper.core.Dataset,
                                        overwrite=True)

In [4]:
with h5tbx.File() as h5:
    h5.create_dataset('data', [1, 2, 3])
    h5.data.software = ('MySoftware', 'v1.1.3')
    h5.data.attrs.sdump(show_private=False)
    sftw = h5.data.software

Attributes of "/data":
----------------------
software:  Software(name='MySoftware', vers='v1.1.3')


Retrieving the software standard attribute will return the Software class:

In [5]:
with h5tbx.File(h5.hdf_filename) as h5:
    sftw = h5.data.software
sftw

Software(name='MySoftware', vers='v1.1.3')

As `software` is an actual HDF5 attribute, we can access the attribute in the usual way and still get the object of type `Software` back. This is because of the package configuration which is per default set so, that the stndard attributes are exposed to the HDF5 Attribute Manager (we'll overwrite the config for clarity and after the example change it)

In [6]:
h5tbx.config.expose_user_prop_to_attrs = True

with h5tbx.File(h5.hdf_filename) as h5:
    sftw = h5.data.attrs['software']
sftw

Software(name='MySoftware', vers='v1.1.3')

Now, let's not expose the standard attributes to the Attribute Mangager. We'll expect to receive the raw attribute string:

In [7]:
h5tbx.config.expose_user_prop_to_attrs = False

with h5tbx.File(h5.hdf_filename) as h5:
    sftw = h5.data.attrs['software']
sftw

{'name': 'MySoftware', 'vers': 'v1.1.3'}

As the software attribute was only registered for datasets and groups, we should not be able to assign a software to a `core.File` object. Note, that python let's you set the property, however, it has not the effect as above because we registered `SoftwareAttribute` only for datasets!

In [8]:
with h5tbx.File() as h5:
    h5.software = ('MySoftware', 'v1.1.3')
    h5.attrs.sdump() # no attribute set as expected

Attributes of "/":
------------------
__h5rdmtoolbox_version__:  0.3.0a8
